In [1]:
from skimage import io ,transform ,feature,measure,filters,exposure
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import os
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
import glob
from commonfunctions import *
import csv
from matplotlib import cm
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin,area_closing
from PIL import Image  
import PIL  
import scipy.misc
import sys
import cv2
np.set_printoptions(threshold=sys.maxsize)
from collections import Counter 

from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.util import img_as_ubyte

In [2]:
#-------------------------------------------------------BINARIZATION-------------------------------------------------------#

def to_binary(img,val):
    threshold = np.copy(img)
    threshold[threshold<val]=0
    threshold[threshold>=val]=1
    return threshold

#--------------------------------------------------------------------------------------------------------------------------#

def binarize(img):
    
    threshold = filters.threshold_otsu(img)
    img = to_binary(img,threshold)
    return img

In [3]:
def countBlack (image):
    blacks = 0
    for color in image.flatten():
        if color == 0:
            blacks += 1
    return blacks

In [4]:
def zooning(img,div):
    img_arr=[]
    width=img.shape[0]
    height=img.shape[1]
    
    for i in range(div):
        for j in range(div):   
            img_arr.append(img[i*width//div : (i+1)*width//div ,j*height//div : (j+1)*height//div ])

    return img_arr

In [5]:
def getVolume(img):
    nrows=img.shape[0]
    ncols=img.shape[1]
    
    area=nrows*ncols
    black_area=countBlack(img)
    volume=black_area/area
    
    return volume

In [6]:
def train(folder):
    feature_vector_all=[]
    direcs = glob.glob ("dataset/*")
    for direc in direcs:
        print(direc)
        files = glob.glob (direc+'/*')
        for file in files:
            #reading an imag
            print("-----------------------",file,"------------------------------")
            img=io.imread(file)

            #converting it to a gray image
            gray_scale_img=rgb2gray(img)
            max_value=np.max(gray_scale_img)
            min_value=np.min(gray_scale_img)
            print(max_value)
            print(min_value)
            if(max_value==1):
                gray_scale_img=(gray_scale_img*255).astype(np.uint8)
            else:
                gray_scale_img=gray_scale_img.astype(np.uint8)




            #binarization process
            note = binarize(gray_scale_img)

            feature_vector=[]
            feature_vector.append(int(direc.split('\\')[1]))

            nrows=note.shape[0]
            ncols=note.shape[1]
            feature_vector.append(nrows)
            feature_vector.append(ncols)

            area=nrows*ncols
            feature_vector.append(area)

            aspect_ratio=ncols/nrows
            feature_vector.append(aspect_ratio)


            black_area=countBlack(note)
            feature_vector.append(black_area)



            volume=getVolume(note)
            feature_vector.append(volume)



            m = measure.moments(1-note,order=3)
            centroid = (m[0, 1] / m[0, 0], m[1, 0] / m[0, 0])
    #         mu = measure.moments_central(1-note, centroid)
    #         moments=measure.moments_normalized(mu)
            feature_vector.append(centroid[0]/ncols)
            feature_vector.append(centroid[1]/nrows)
            mom=cv2.HuMoments(cv2.moments(1-note)).flatten()
            for mm in mom:
                feature_vector.append(mm)



            img_array=zooning(note,4)
            for i in range(len(img_array)):
                feature_vector.append(getVolume(img_array[i]))



    #         show_images([gray_scale_img,note],['image','after binarization']) 
            feature_vector_all.append(feature_vector)


    # opening the csv file in 'w+' mode 
    file = open('dataset.csv', 'w+', newline ='') 

    # writing the data into the file 
    with file:
            write = csv.writer(file) 
            write.writerows(feature_vector_all) 


    return True

In [7]:

feature_vector_all=[]
direcs = glob.glob ("dataset/*")
for direc in direcs:
    print(direc)
    files = glob.glob (direc+'/*')
    for file in files:
        #reading an imag
        print("-----------------------",file,"------------------------------")
        img=io.imread(file)
        
        #converting it to a gray image
        gray_scale_img=rgb2gray(img)
        max_value=np.max(gray_scale_img)
        min_value=np.min(gray_scale_img)
        print(max_value)
        print(min_value)
        if(max_value==1):
            gray_scale_img=(gray_scale_img*255).astype(np.uint8)
        else:
            gray_scale_img=gray_scale_img.astype(np.uint8)
            
            


        #binarization process
        note = binarize(gray_scale_img)
        
        feature_vector=[]
        feature_vector.append(int(direc.split('\\')[1]))
      
        nrows=note.shape[0]
        ncols=note.shape[1]
        feature_vector.append(nrows)
        feature_vector.append(ncols)
        
        area=nrows*ncols
        feature_vector.append(area)

        aspect_ratio=ncols/nrows
        feature_vector.append(aspect_ratio)


        black_area=countBlack(note)
        feature_vector.append(black_area)



        volume=getVolume(note)
        feature_vector.append(volume)
       


        m = measure.moments(1-note,order=3)
        centroid = (m[0, 1] / m[0, 0], m[1, 0] / m[0, 0])
#         mu = measure.moments_central(1-note, centroid)
#         moments=measure.moments_normalized(mu)
        feature_vector.append(centroid[0]/ncols)
        feature_vector.append(centroid[1]/nrows)
        mom=cv2.HuMoments(cv2.moments(1-note)).flatten()
        for mm in mom:
            feature_vector.append(mm)



        img_array=zooning(note,4)
        for i in range(len(img_array)):
            feature_vector.append(getVolume(img_array[i]))
        
        
    
#         show_images([gray_scale_img,note],['image','after binarization']) 
        feature_vector_all.append(feature_vector)


# opening the csv file in 'w+' mode 
file = open('dataset.csv', 'w+', newline ='') 
  
# writing the data into the file 
with file:
        write = csv.writer(file) 
        write.writerows(feature_vector_all) 
    



dataset\1
----------------------- dataset\1\00018-13.jpg ------------------------------
255
0
----------------------- dataset\1\00037-13.jpg ------------------------------
255
0
----------------------- dataset\1\00108-13.jpg ------------------------------
255
0
----------------------- dataset\1\00149-13.jpg ------------------------------
255
0
----------------------- dataset\1\00161-13.jpg ------------------------------
255
0
----------------------- dataset\1\00192-13.jpg ------------------------------
255
0
----------------------- dataset\1\00292-13.jpg ------------------------------
255
0
----------------------- dataset\1\00303-13.jpg ------------------------------
255
0
----------------------- dataset\1\00379-13.jpg ------------------------------
255
0
----------------------- dataset\1\00399-13.jpg ------------------------------
255
0
----------------------- dataset\1\00622-13.jpg ------------------------------
255
0
----------------------- dataset\1\00637-13.jpg -------------------

<ipython-input-7-1f673aa44f42>:12: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  gray_scale_img=rgb2gray(img)


 dataset\1\00834-13.jpg ------------------------------
255
0
----------------------- dataset\1\00846-13.jpg ------------------------------
255
0
----------------------- dataset\1\00867-13.jpg ------------------------------
255
0
----------------------- dataset\1\00916-13.jpg ------------------------------
255
0
----------------------- dataset\1\00978-13.jpg ------------------------------
255
0
----------------------- dataset\1\00979-13.jpg ------------------------------
255
0
----------------------- dataset\1\01024-13.jpg ------------------------------
255
0
----------------------- dataset\1\01222-13.jpg ------------------------------
255
0
----------------------- dataset\1\01279-13.jpg ------------------------------
255
0
----------------------- dataset\1\01377-13.jpg ------------------------------
255
0
----------------------- dataset\1\01383-13.jpg ------------------------------
255
0
----------------------- dataset\1\01407-13.jpg ------------------------------
255
0
---------------

<ipython-input-7-1f673aa44f42>:12: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  gray_scale_img=rgb2gray(img)


255
0
----------------------- dataset\10\00337-11.jpg ------------------------------
255
0
----------------------- dataset\10\00339-27.jpg ------------------------------
255
0
----------------------- dataset\10\00353-27.jpg ------------------------------
255
0
----------------------- dataset\10\00363-27.jpg ------------------------------
255
0
----------------------- dataset\10\00381-11.jpg ------------------------------
255
0
----------------------- dataset\10\00387-11.jpg ------------------------------
255
0
----------------------- dataset\10\00426-11.jpg ------------------------------
255
0
----------------------- dataset\10\00445-27.jpg ------------------------------
255
0
----------------------- dataset\10\00486-27.jpg ------------------------------
255
0
----------------------- dataset\10\00525-11.jpg ------------------------------
255
0
----------------------- dataset\10\00527-11.jpg ------------------------------
255
0
----------------------- dataset\10\00534-27.jpg -----------

<ipython-input-7-1f673aa44f42>:12: FutureWarning: Non RGB image conversion is now deprecated. For RGBA images, please use rgb2gray(rgba2rgb(rgb)) instead. In version 0.19, a ValueError will be raised if input image last dimension length is not 3.
  gray_scale_img=rgb2gray(img)


----------------------- dataset\10\5432.png ------------------------------
1.0
0.0
----------------------- dataset\10\6.jpg ------------------------------
1.0
0.0
----------------------- dataset\10\98765.png ------------------------------
255
0
----------------------- dataset\10\IMG-20150411-WA0000.jpg ------------------------------
1.0
0.0
dataset\11
----------------------- dataset\11\00009-6.jpg ------------------------------
255
0
----------------------- dataset\11\00045-24.jpg ------------------------------
255
0
----------------------- dataset\11\00060-24.jpg ------------------------------
255
0
----------------------- dataset\11\00062-6.jpg ------------------------------
255
0
----------------------- dataset\11\00113-6.jpg ------------------------------
255
0
----------------------- dataset\11\00135-6.jpg ------------------------------
255
0
----------------------- dataset\11\00182-24.jpg ------------------------------
255
0
----------------------- dataset\11\00198-24.jpg -------

<ipython-input-7-1f673aa44f42>:54: RuntimeWarning: invalid value encountered in double_scalars
  centroid = (m[0, 1] / m[0, 0], m[1, 0] / m[0, 0])


----------------------- dataset\13\119041.png ------------------------------
1.0
0.0
----------------------- dataset\13\1198.jpg ------------------------------
1.0
0.0
----------------------- dataset\13\1211.jpg ------------------------------
1.0
0.0
----------------------- dataset\13\1237.jpg ------------------------------
1.0
0.0
----------------------- dataset\13\1432.png ------------------------------
1.0
0.0
----------------------- dataset\13\2.JPG ------------------------------
1.0
0.0
----------------------- dataset\13\2154.jpg ------------------------------
1.0
0.0
----------------------- dataset\13\221.png ------------------------------
255
0
----------------------- dataset\13\2211.jpg ------------------------------
1.0
0.0
----------------------- dataset\13\2211.png ------------------------------
255
0
----------------------- dataset\13\22113.png ------------------------------
1.0
0.0
----------------------- dataset\13\221133.png ------------------------------
1.0
0.0
-------

In [8]:
#but first lets try on one template 
#features:
#1-area
#2-aspect ratio
#3-black area
#4-compactness
#5-moments
#6-ncols
#7-volume16regions
#8-skelton features
#9-top bottom

